In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [31]:
feats_import = pd.read_csv('All_Data_2006_2016.csv', index_col=0)

In [104]:
feats = feats_import.drop(['Season', 'Gameday', 'TID_H', 'TID_A'], axis=1).drop('FTGD', axis=1)
feats.loc[:, 'HTGD'] = feats.loc[:, 'HTGD'] - feats.loc[:, 'HTGD'].min()
feats = feats/feats.apply(max)

GD_min = int(feats_import.loc[:, 'FTGD'].min())
GD_max = int(feats_import.loc[:, 'FTGD'].max())
label = feats_import.loc[:, 'FTGD'] - GD_min

GD_spread = GD_max-GD_min + 1
ID = np.eye(GD_spread)

X = feats.iloc[0:3000].as_matrix()
y_pre = map(int, label.ix[0:3000].as_matrix())

y = np.array([ID[i] for i in y_pre])

In [123]:
def baseline_model():
    model = Sequential()
    model.add(Dense(len(X[0]), input_dim=len(X[0]), init='uniform', activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [124]:
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=20, batch_size=50, verbose=0)

In [125]:
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, X, y, cv=kfold)
print "Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100)

In [118]:
X_test = feats.iloc[3001:].as_matrix()
y_test_pre = map(int, label.iloc[3001:].as_matrix())

y_test = np.array([ID[i] for i in y_test_pre])

In [119]:
oos = cross_val_score(estimator, X_test, y_test, cv=kfold)

In [120]:
print "Baseline: %.2f%% (%.2f%%)" % (oos.mean()*100, oos.std()*100)

Baseline: 0.00% (0.00%)
